<a href="https://colab.research.google.com/github/Roni81/smartfarm/blob/main/growing1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 24 20:38:08 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Conv2D, MaxPooling2D, Flatten, LSTM, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [5]:
# 데이터셋 경로 설정
image_paths = glob("/content/drive/MyDrive/growing2_temp/growing2_temp/images/*.jpg")
metadata_path = "/content/drive/MyDrive/growing2_temp/growing2_temp/metas/*.csv"
labels_path = "/content/drive/MyDrive/growing2_temp/growing2_temp/combined_dataset.csv"


In [6]:
# 이미지 데이터 로드 및 전처리
def load_images(image_paths):
    images = [cv2.imread(img_path) for img_path in image_paths]
    images = np.array(images) / 255.0  # 정규화
    return images

# 메타데이터 및 라벨 데이터 로드
def load_metadata_and_labels(metadata_path, labels_path):
    metadata = pd.read_csv(metadata_path)
    labels = pd.read_csv(labels_path)
    return metadata, labels

In [ ]:
# 이미지, 메타데이터, 라벨 로드
images = load_images(image_paths)
metadata, labels = load_metadata_and_labels(metadata_path, labels_path)


<ipython-input-6-21cc06f1c1bb>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images = np.array(images) / 255.0  # 정규화


CNN RNN 앙상블

In [ ]:
# 이미지 CNN 모델
image_input = Input(shape=(height, width, channels))
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
maxpool1 = MaxPooling2D((2, 2))(conv1)
flatten1 = Flatten()(maxpool1)

# 메타데이터 RNN 모델
meta_input = Input(shape=(num_metadata_features,))
embedding = Embedding(input_dim=num_categories, output_dim=embedding_dim)(meta_input)
lstm1 = LSTM(64)(embedding)

# 모델 병합
merged = concatenate([flatten1, lstm1])
dense1 = Dense(128, activation='relu')(merged)
output = Dense(num_classes, activation='softmax')(dense1)

model = Model(inputs=[image_input, meta_input], outputs=output)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit([X_image_train, X_meta_train],
          to_categorical(y_train),
          epochs=10, batch_size=32,
          validation_data=([X_image_val, X_meta_val], to_categorical(y_val)))

In [ ]:
predictions = model.predict([X_image_val, X_meta_val])

CNN_LSTM

In [ ]:
# 이미지 CNN 모델
image_input = Input(shape=(height, width, channels))
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
maxpool1 = MaxPooling2D((2, 2))(conv1)
flatten1 = Flatten()(maxpool1)

# 메타데이터 LSTM 모델
meta_input = Input(shape=(num_metadata_features,))
embedding = Embedding(input_dim=num_categories, output_dim=embedding_dim)(meta_input)
lstm1 = LSTM(64)(embedding)

# 모델 병합
merged = concatenate([flatten1, lstm1])
dense1 = Dense(128, activation='relu')(merged)
output = Dense(num_classes, activation='softmax')(dense1)

model = Model(inputs=[image_input, meta_input], outputs=output)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit([X_image_train, X_meta_train],
          to_categorical(y_train),
          epochs=10, batch_size=32,
          validation_data=([X_image_val, X_meta_val], to_categorical(y_val)))

Multi_Modal Neural Network